# Monday Afternoon: Excursion Effects

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.linear_model import LinearRegression

np.random.seed(1)

## Simple Data Generating Environment
- "Covariates": $X_t = (1, B_t, C_t, D_t)$ where $B_t$ is binary and $C_t$ is continuous and $D_t$ is a measure of "dosage"
    - Specifically $D_t = \frac{1}{1-\gamma} \sum_{t'=1}^{t-1} \gamma^{t'} A_{t'}$ for $\gamma = 0.95$. We normalize by $1-\gamma$ to ensure $D_t \in [0,1]$.
- Binary action $A_t \in \{0, 1\}$
- Rewards are generated as follows:

$$
R_{t+1} = f_0(X_t)^\top \alpha_0 + A_t f_1(X_t)^\top \alpha_1 + \epsilon_t
$$

where $f_0(X_t) = (1, B_t, C_t, D_t)$, $f_1(X_t) = (1, C_t)$, 


$\alpha_0 = (1, -0.8, 0.05, -0.7)$, $\alpha_1 = (0.3, 0)$, $\epsilon_t \sim \mathcal{N}(0,\sigma_{\mathrm{env}}^2)$.

In [ ]:
env_params = {
    "alpha0": np.array([1, 0.5, 0.2, -0.7]),
    "alpha1": np.array([0.3, 0.05]),
    "sigma_env": 0.5,
    "B_p": 0.3,
}

def generate_reward(state, action, env_params):
    base_state = np.array([1, state[1], state[2], state[3]])
    treat_state = np.array([1, state[2]])
    
    base_mean_reward = np.dot( base_state, env_params["alpha0"] )
    mean_reward = base_mean_reward + \
                    action * np.dot( treat_state, env_params["alpha1"] )
    reward = mean_reward + np.random.normal(scale=env_params["sigma_env"])
    return reward
        

def generate_state(prev_state, prev_action, prev_reward, env_params):
    B = np.random.binomial(1, env_params["B_p"])
    C = np.random.normal(3, scale=1)
    
    # Form dosage update
    gamma = 0.95
    norm_gamma = 1/(1-gamma)
    if prev_state is None:
        dosage = np.random.binomial(1, 0.5)
    else:
        dosage = (gamma*norm_gamma*prev_state[-1] + prev_action ) / norm_gamma
    
    state = np.array([1, B, C, dosage])
    return state

## Simple Randomization Policy

In [ ]:
class SimplePolicy:
    
    def __init__(self):
        pass
      
    
    def form_action1_prob(self, state, user_id):
        # State variables order: "intercept", "binary", "continuous"
        
        if state[1] == 1: # binary state variable
            return 0.3
        else:
            return 0.5
        
    def update_algorithm(self, new_states, new_actions, new_rewards, all_user_ids):
        pass

### Preparing the MRT Dataframe

In [ ]:
def make_empty_study_df(T, n):
    dataset_rownames = ["user_id", "decision_t", "reward", 
                        "action", "action1_prob", "intercept", "binary", "continuous", "dosage"]

    # Fill in user_ids and decision times
    user_id_all = np.repeat([i for i in range(1,n+1)], T)
    decision_t_all = np.tile([i for i in range(1,T+1)], n)
    empty_cols = np.empty((n*T, len(dataset_rownames)-2))
    empty_cols[:] = np.nan

    # Make dataframe to record study data
    dataset_entries = np.hstack( [ np.stack([user_id_all, decision_t_all]).T, empty_cols ] )
    study_df = pd.DataFrame(dataset_entries, columns=dataset_rownames)

    # Change type of columns
    type_dict = {
        'user_id': int,
        'decision_t': int,
    }
    study_df = study_df.astype(type_dict)
    
    return study_df

### Simulating the MRT

In [ ]:
def simulate_MRT(T, n, env_params, RL_alg):
    study_df = make_empty_study_df(T, n)
    
    # Loop over decision times
    for t in range(1,T+1):
    
        # Loop over users in the study
        all_states = []
        all_actions = []
        all_action1_probs = []
        all_rewards = []
        for user_id in range(1,n+1):
        
            # Generate state
            if t == 1:
                state = generate_state(None, None, None, env_params)
            else:
                state = generate_state(prev_states[user_id-1], 
                                    prev_actions[user_id-1], 
                                    prev_rewards[user_id-1], env_params)
        
            # Form action selection probabilities
            action1_prob = RL_alg.form_action1_prob(state, user_id)
            action = np.random.binomial(1, action1_prob)
        
            # Generate reward
            reward = generate_reward(state, action, env_params)
        
            # Record data
            all_states.append(state)
            all_actions.append(action)
            all_action1_probs.append(action1_prob)
            all_rewards.append(reward)
    
        # Save all user data
        all_states = np.array(all_states)
        all_actions = np.array(all_actions)
        all_action1_probs = np.array(all_action1_probs)
        all_rewards = np.array(all_rewards)
        
        # Update Algorithm
        RL_alg.update_algorithm(new_states = all_states, 
                     new_actions = all_actions, 
                     new_rewards = all_rewards, 
                     all_user_ids = np.arange(1,n+1))
    
        idx_t = study_df.index[study_df['decision_t'] == t]
        half_row_data = np.vstack([all_rewards, all_actions, all_action1_probs]).T
        row_data = np.hstack([half_row_data, all_states])      
        study_df.iloc[idx_t,2:] = row_data
        
        # Prepare for next decision time
        prev_states = all_states
        prev_actions = all_actions
        prev_rewards = all_rewards
    

    type_dict = {
        'action': int,
        'intercept': int,
        'binary': int,
    }
    study_df = study_df.astype(type_dict)
    return study_df

### Simulate MRT

In [ ]:
T = 50
n = 100

# Form Decision Making Policy and Run MRT Study
policy = SimplePolicy()
study_df = simulate_MRT(T, n, env_params, policy)


# Print first 10 rows of dataframe
print("Average Treatment Prob: {}".format(np.mean(study_df['action1_prob'])))
print("")
study_df.head(10)

## (1) Estimating Causal Excursion Effect 
$$
\psi_1(X_{i,t})^\top \theta^\star = \mathbb{E} \left[ Y_{t+1}( \bar A_{t-1}, 1 ) - Y_{t+1}( \bar A_{t-1}, 0 ) \big| H_{i,t-1}, X_{i,t} \right]
$$

$$
\hat{\theta} = \mathrm{argmin_{(\theta_0, \theta_1) \in \mathbb{R}^2}} \left\{ \frac{1}{n} \sum_{i=1}^n \sum_{t=1}^T \left( Y_{i,t+1} - \psi_0(H_{i,t-1}, X_{i,t})^\top \eta - (A_{i,t}-\pi_{i,t}) \psi_1(X_{i,t})^\top \theta \right)^2 \right\}
$$
where $\psi_0(H_{i,t-1}, X_{i,t}) = [1, B_{i,t}, C_{i,t}]$ and $\psi_1(X_{i,t}) = [1, C_{i,t}]$. Recall that $X_{i,t} = (1, B_{i,t}, C_{i,t})$.

In [ ]:
def form_feat_matrix(study_df, base_feat_names, treat_feat_names, 
                     action_centering=False):
    """
    If action_centering = True
        Return matrix where each row is [ g(H_{i,t-1}, X_{i,t}), (A_{i,t}-\pi_{i,t}) f(X_{i,t}) ]
    Else:
        Return matrix where each row is [ g(H_{i,t-1}, X_{i,t}), A_{i,t} f(X_{i,t}) ]
    """
    base_feats = np.vstack([study_df[feat] for feat in base_feat_names]).T
    treat_feats = np.vstack([study_df[feat] for feat in treat_feat_names]).T
    actions = actions = study_df['action']
    action_probs = study_df['action1_prob']
    
    if action_centering:
        """
        EXERCISE: Form features with action centering
        """
        pass
    else:
        actions = np.expand_dims(actions, 1)
        feat_matrix = np.concatenate([ base_feats, actions*treat_feats], axis=1)
        
    return feat_matrix

In [ ]:
def fit_linear_model(feat_matrix, Y_vec, weights=None):
    reg = LinearRegression().fit(feat_matrix, Y_vec, weights)
    thetahat = reg.coef_.copy()
    thetahat[0] = reg.intercept_
    return thetahat

In [ ]:
# Form Least Squares Estimator
Y_vec = study_df['reward']
feat_matrix = form_feat_matrix(study_df, 
                               base_feat_names = ["intercept", "binary", "continuous"],
                               treat_feat_names = ["intercept", "continuous"],
                               action_centering=True)

# Fit Linear Model
thetahat = fit_linear_model(feat_matrix, Y_vec)

print("True excursion effect (theta):")
print( env_params['alpha1'] )
print("Estimated excursion effect (theta):")
print( thetahat[-2:] )

## (2) Investiagate the Impact of Action Centering
$$
\hat{\theta} = \mathrm{argmin_{(\theta_0, \theta_1) \in \mathbb{R}^2}} \left\{ \frac{1}{n} \sum_{i=1}^n \sum_{t=1}^T \left( Y_{i,t+1} - \psi_0(H_{i,t-1}, X_{i,t})^\top \eta - (A_{i,t}-\pi_{i,t}) \psi_1(X_{i,t})^\top \theta \right)^2 \right\}
$$
where $\psi_0(H_{i,t-1}, X_{i,t}) = [1, B_{i,t}, C_{i,t}]$ and $\psi_1(X_{i,t}) = [1, C_{i,t}]$. Recall that $X_{i,t} = (1, B_{i,t}, C_{i,t})$.

__Questions:__
- What happens if the for the average reward is very bad? For example $\psi_0(H_{i,t-1}, X_{i,t}) = 1$?
- How does this impact the estimation of the excursion effect theta? Try both with and without action centering

In [ ]:
# Form Least Squares Estimator no action centering #################
"""
EXERCISE: Fit Linear Model without action centering
"""
thetahat_noac = np.nan(5)


# Form Least Squares Estimator with action centering #################
"""
# EXERCISE: Fit Linear Model with action centering 
"""
thetahat_ac = np.zeros(5)




print("True excursion effect (theta):")
print( env_params['alpha1'] )
print("Estimated excursion effect - no action centering (theta):")
print( thetahat_noac[-2:] )
print("Estimated excursion effect - with action centering (theta):")
print( thetahat_ac[-2:] )

## (3) Generate MRT Dataset with a Reinforcement Learning Algorithm
- Use the following posterior sampling algorithm in a simulated MRT
    - The algorithm must only learn using the data of a single user (individual RL algorithms).
- Alternatively you can code your own RL Algorithm of choice (e.g. from those Raaz taught earlier)
    - Make sure to restrict the action selection probabilities within [$\pi_\min, 1-\pi_\min$]
    - Specifically, fill out the `form_action1_prob` and `update_algorithm` function

### Example RL algorithm you could implement: Posterior Sampling
- Normal priors and likelihoods
- RL algorithm's model of the reward:

$$
R_{t+1} = \phi_0(X_t)^\top \beta_0 + A_t \phi_1(X_t)^\top \beta_1 + \epsilon_t
$$

where $\phi_0(X_t) = (1, C_t)$, $\phi_1(X_t) = (1, C_t)$, and $\epsilon_t \sim \mathcal{N}(0,\sigma^2_{\mathrm{alg}})$.

- Prior: 
$$
\begin{pmatrix}
    \beta_0 \\ 
    \beta_1
\end{pmatrix} \sim \mathcal{N} \left( \begin{bmatrix} 0.5 \\ 0 \\ 0 \\ 0 \end{bmatrix}, I_4 \right)
$$

- Constrain action selection probabilities so for $\pi_{\min} = 0.1$

$$
\mathbb{P}(A_t = 1 | H_t, X_t) \in [\pi_{\min}, 1 - \pi_{\min}] ~~~~ \mathrm{with~probability}~1
$$

In [ ]:
class PosteriorSampler:
    
    def __init__(self, n, prior_mean, prior_var, sigma_alg, pi_min):
        self.sigma_alg = sigma_alg
        self.pi_min = pi_min
        
        all_posteriors = {}
        for user_id in range(1, n+1):
            posterior = {
                "posterior_mean": prior_mean.copy(),
                "posterior_var": prior_var.copy(),
            }
            all_posteriors[user_id] = posterior
            
        self.all_posteriors = all_posteriors
      
    
    def form_action1_prob(self, state, user_id):
    
        # Posterior Mean and Variance for beta_1 (treatment effect)
        posterior = self.all_posteriors[user_id]
        post_mean_beta1 = posterior['posterior_mean'][-1]
        post_var_beta1 = posterior['posterior_var'][-1,-1]
    
        raw_prob = sp.stats.norm.cdf( post_mean_beta1 / np.sqrt(post_var_beta1) )
    
        # Restrict action selection probabilities to [pi_min, 1-pi_min]
        return self.clip_action1_prob(raw_prob, self.pi_min)
    
    
    def clip_action1_prob(self, raw_prob, pi_min):
        clipped_prob = np.maximum(pi_min, raw_prob)
        clipped_prob = np.minimum(1-pi_min, clipped_prob)
        return clipped_prob
        
        
    def update_algorithm(self, new_states, new_actions, new_rewards, all_user_ids):
        
        # update posterior for each user
        for user_id in all_user_ids:
            user_state = new_states[user_id-1]
            user_action = new_actions[user_id-1]
            user_reward = new_rewards[user_id-1]
        
            posterior = self.all_posteriors[user_id]
            post_var = posterior['posterior_var']
            post_mean = posterior['posterior_mean']
            feat_vec = np.array([user_state[0], user_state[2], 
                                 user_action, user_reward*user_state[2]])

            inv_post_var = np.linalg.inv(post_var)
            new_inv_post_var = inv_post_var + np.outer(feat_vec, feat_vec) / self.sigma_alg
            new_post_var = np.linalg.inv( new_inv_post_var )
    
            new_post_mean_num = np.matmul(inv_post_var, post_mean) + \
                                user_reward * feat_vec / self.sigma_alg
            new_post_mean = np.matmul( new_post_var, new_post_mean_num )
    
            self.all_posteriors[user_id]['posterior_mean'] = new_post_mean
            self.all_posteriors[user_id]['posterior_var'] = new_post_var

In [ ]:
class RLAlg:
    
    def __init__(self, n, pi_min):
        self.pi_min = pi_min
        self.n
        
    
    def form_action1_prob(self, state, user_id):
        """
        TODO: form action selection probability for a given user and state
        """
        # State variables order: "intercept", "binary", "continuous"
        
        if state[1] == 1: # binary state variable
            raw_prob = 0.3
        else:
            raw_prob = 0.5
    
        # Restrict action selection probabilities to [pi_min, 1-pi_min]
        return self.clip_action1_prob(raw_prob, self.pi_min)
    
    
    def clip_action1_prob(self, raw_prob, pi_min):
        """
        Constrain action selection probabilities between pi_min and 1-pi_min
        """
        assert 0 < pi_min <= 0.5
        clipped_prob = np.maximum(pi_min, raw_prob)
        clipped_prob = np.minimum(1-pi_min, clipped_prob)
        return clipped_prob
        
        
    def update_algorithm(self, new_states, new_actions, new_rewards, all_user_ids):
        """
        TODO: Update RL algorithm
        """
        pass

### Generate Dataset with RL Algorithm

In [ ]:
# RL Algorithm Hyperparameters
pi_min = 0.05
prior_mean = np.array([0.5, 0, 0, 0])
prior_var = np.eye(4)
sigma_alg = 1


# Form RL Algorithm and Run MRT Study
RL_alg = PosteriorSampler(n, prior_mean, prior_var, sigma_alg, pi_min)
study_df_RL = simulate_MRT(T, n, env_params, RL_alg)


# Print first 10 rows of dataframe
print("Average Treatment Prob: {}".format(np.mean(study_df_RL['action1_prob'])))
print("")
study_df_RL.head(10)

## (4) Estimating Causal Excursion Effect on RL Data
$$
\theta^\star = \mathbb{E} \left[ Y_{t+1}( \bar A_{t-1}, 1 ) - Y_{t+1}( \bar A_{t-1}, 0 ) \right]
$$

$$
\hat{\theta} = \mathrm{argmin_{(\theta_0, \theta_1) \in \mathbb{R}^2}} \left\{ \frac{1}{n} \sum_{i=1}^n \sum_{t=1}^T W_{i,t} \left( Y_{i,t+1} - \psi_0(H_{i,t-1}, X_{i,t})^\top \eta - (A_{i,t}-\pi_{i,t}) \psi_1(X_{i,t})^\top \theta \right)^2 \right\}
$$
where $\psi_0(H_{i,t-1}, X_{i,t}) = [1, B_{i,t}, C_{i,t}]$ and $\psi_1(X_{i,t}) = [1, C_{i,t}]$. Recall that $X_{i,t} = (1, B_{i,t}, C_{i,t})$.

__Make sure to include weights $W_{i,t}$!!__
$$W_{i,t} = \bigg( \frac{p(X_{i,t})}{\pi_{i,t}} \bigg)^{A_{i,t}} \bigg( \frac{1-p(X_{i,t})}{1-\pi_{i,t}} \bigg)^{1-A_{i,t}}$$
for some pre-specified policy $p$, to ensure that $p(X_{i,t})$ only depends on $X_{i,t}$.

In [ ]:
# Form Least Squares Estimator
Y_vec = study_df['reward']
feat_matrix = form_feat_matrix(study_df, 
                               base_feat_names = ["intercept", "binary", "continuous"],
                               treat_feat_names = ["intercept", "continuous"],
                               action_centering=True)

"""
EXERCISE: Form Vector of weights
"""
action1_probs = study_df_RL['action1_prob'].to_numpy()
actions = study_df_RL['action'].to_numpy()
weight_vec = np.ones(action1_probs.shape) # EXERCISE: change this to weights W_{i,t}
thetahat = fit_linear_model(feat_matrix, Y_vec, weight_vec)

print("True excursion effect (theta):")
print( env_params['alpha1'] )
print("Estimated excursion effect (theta):")
print( thetahat[-2:] )

## (5) Additional Exploration (on your own time)
- What happens if you change $\pi_\min$? How do we expect our estimation of the excursion effect to change?
- Try estimate a marginal excursion effect (not conditional on state)?
- Estimate the limiting variance of the excursion effect and form a confidence interval via a normal approximation. For the variance formula see Proposition 3.1 in [Assessing Time-Varying Causal Effect Moderation in Mobile Health](https://arxiv.org/abs/1601.00237).
- You could form a more complex reward generating model. In this setting, one could use a machine learning model (e.g. random forrest) to replace $\psi_1(H_{i,t-1}, X_{i,t})^\top \eta$. the See [Double/Debiased Machine Learning for Treatment and Causal Parameters](https://arxiv.org/abs/1608.00060).